In [8]:
import pandas as pd

# Cargar el dataset original
df = pd.read_csv("edges_info.csv", sep=";")

columnas_si = [
    "u", "v", 
    "Fallecidos", "Graves", "Menos_Grav", "Leves", "Ilesos",

    # Estado de la calzada
    "Estado_Cal__MALO", "Estado_Cal__REGULAR",

    # Tipo de calzada
    "Tipo_Calza__ADOQUIN", "Tipo_Calza__ASFALTO",
    "Tipo_Calza__CONCRETO", "Tipo_Calza__MIXTO",
    "Tipo_Calza__RIPIO", "Tipo_Calza__TIERRA",

    # Tipo de vía (Calzada)
    "Calzada__BIDIRECCIONAL",
    "Calzada__BIDIRECCIONAL CON BANDEJON",
    "Calzada__UNIDIRECCIONAL",

    # Zona
    "Zona__RURAL", "Zona__URBANA",

    # Ubicación
    "Ubicación__ACERA O BERMA",
    "Ubicación__CRUCE CON SEMAFORO APAGADO",
    "Ubicación__CRUCE CON SEMAFORO FUNCIONANDO",
    "Ubicación__CRUCE CON SEÑAL CEDA EL PASO",
    "Ubicación__CRUCE CON SEÑAL PARE",
    "Ubicación__CRUCE REGULADO POR CARABINERO",
    "Ubicación__CRUCE SIN SEÑALIZACION",
    "Ubicación__ENLACE A DESNIVEL",
    "Ubicación__ENLACE A NIVEL",
    "Ubicación__OTROS NO CONSIDERADOS",
    "Ubicación__PLAZA DE PEAJE",
    "Ubicación__PUENTE",
    "Ubicación__ROTONDA",
    "Ubicación__TRAMO DE VIA CURVA HORIZONTAL",
    "Ubicación__TRAMO DE VIA CURVA VERTICAL",
    "Ubicación__TRAMO DE VIA RECTA",
    "Ubicación__TUNEL",

    # Tipos de carretera
    "hw_busway", "hw_living_street", "hw_motorway", "hw_motorway_link",
    "hw_primary", "hw_primary_link", "hw_residential",
    "hw_secondary", "hw_secondary_link", "hw_tertiary",
    "hw_tertiary_link", "hw_trunk", "hw_unclassified",

    # Límites de velocidad
    "ms_10", "ms_100", "ms_120", "ms_15", "ms_20",
    "ms_30", "ms_35", "ms_40", "ms_5", "ms_50",
    "ms_60", "ms_70", "ms_80",

    # Longitud
    "length_m"
]


# Filtrar dataset
df_nuevo = df[columnas_si]

# Guardar el dataset
df_nuevo.to_csv("edges_nuevo.csv", index=False)



Ahora definimos una nueva columna que mezcle info de las otras:
* Estado_calzada: 

0 si y solo si la columna Estado_Cal__Regular es 1

1 si y solo si Estado_Cal__MALO es 1 

2 en cualquier otro caso (bien).     

In [ ]:
# Crear la columna primero
import numpy as np
df = pd.read_csv("edges_nuevo.csv", sep=",")
df["Estado_calzada"] = np.select(
    [
        df["Estado_Cal__REGULAR"] == 1,
        df["Estado_Cal__MALO"] == 1
    ],
    [
        0,
        1
    ],
    default=2
)

# Reordenar para que quede al lado de las columnas Estado_Cal__
cols = list(df.columns)

# Encontrar índice donde insertar
idx = cols.index("Estado_Cal__REGULAR") + 1

# Reconstruir el orden
new_cols = (
    cols[:idx] +
    ["Estado_calzada"] +
    cols[idx:-1]  # el -1 porque Estado_calzada originalmente está al final
)

df = df[new_cols]
df = df.drop(columns=["Estado_Cal__REGULAR", "Estado_Cal__MALO"])
# Guardar el dataset

df.to_csv("edges_nuevo.csv", index=False)


Convertimos a binarias las columnas Tipo_Calza_*
Y definimos Tipo_calzada:
* 0 si "Tipo_Calza__ADOQUIN" = 1
* 1 si "Tipo_Calza__ASFALTO" = 1
* 2 si "Tipo_Calza__CONCRETO" = 1
* 3 si "Tipo_Calza__MIXTO" = 1
* 4 si "Tipo_Calza__RIPIO" = 1 
* 5 si "Tipo_Calza__TIERRA" = 1
* 3 en el caso que en una misma fila exista más de un tipo

In [12]:
cols_tipo = [
    "Tipo_Calza__ADOQUIN", "Tipo_Calza__ASFALTO", "Tipo_Calza__CONCRETO",
    "Tipo_Calza__MIXTO", "Tipo_Calza__RIPIO", "Tipo_Calza__TIERRA"
]

df[cols_tipo] = (df[cols_tipo] > 0).astype(int)

# Definimos el orden de prioridad según tu numeración
tipo_cols = [
    "Tipo_Calza__ADOQUIN",   # 0
    "Tipo_Calza__ASFALTO",   # 1
    "Tipo_Calza__CONCRETO",  # 2
    "Tipo_Calza__MIXTO",     # 3
    "Tipo_Calza__RIPIO",     # 4
    "Tipo_Calza__TIERRA"     # 5
]

# Obtenemos cuántos tipos aparecen en cada fila
df["cantidad_tipos"] = df[tipo_cols].sum(axis=1)

# Caso especial: si hay más de un tipo -> asignar 3
df["Tipo_calzada"] = df.apply(
    lambda row: 3 if row["cantidad_tipos"] > 1 else row[tipo_cols].idxmax(), 
    axis=1
)

# Convertimos el nombre de columna a número según tu tabla
mapa = {
    "Tipo_Calza__ADOQUIN": 0,
    "Tipo_Calza__ASFALTO": 1,
    "Tipo_Calza__CONCRETO": 2,
    "Tipo_Calza__MIXTO": 3,
    "Tipo_Calza__RIPIO": 4,
    "Tipo_Calza__TIERRA": 5
}

df["Tipo_calzada"] = df["Tipo_calzada"].replace(mapa)

# Eliminamos columna auxiliar opcionalmente
df.drop(columns=["cantidad_tipos"], inplace=True)
# Eliminamos las columaas
df.drop(
    columns=[
        "Tipo_Calza__ADOQUIN",
        "Tipo_Calza__ASFALTO",
        "Tipo_Calza__CONCRETO",
        "Tipo_Calza__MIXTO",
        "Tipo_Calza__RIPIO",
        "Tipo_Calza__TIERRA"
    ],
    inplace=True
)

df.to_csv("edges_nuevo.csv", index=False)

C:\Users\maxil\AppData\Local\Temp\ipykernel_8452\2607429047.py:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Tipo_calzada"] = df["Tipo_calzada"].replace(mapa)


Ahora nos ocupamos de la dirección de la calzada
* Calzada_BIDIRECCIONAL si es mayor que 1 entonces es 1
* Lo mismo con Calzada_BIDIRECCIONAL CON BANDEJON y con Calzada_UNIDIRECCIONAL

In [13]:
cols = [
    "Calzada__BIDIRECCIONAL",
    "Calzada__BIDIRECCIONAL CON BANDEJON",
    "Calzada__UNIDIRECCIONAL"
]
df[cols] = df[cols].clip(upper=1)
df.to_csv("edges_nuevo.csv", index=False)

Ahora creamos una nueva columnas que este al final de Calzada_UNIDIRECCIOANL y que es llame Direccion_calzada y su valor sea:
* 1 siempre que Calzada_UNIDIRECCIONAL = 1 (independiente de las otras calzadas)
* 0 Si calzada_unidireccional = 0

(CUIDADO AQUÍ, CAMBIAR MÁS ADELANTE SI ES NECESARIO). Consideré tener menos calles con una dirección, en vez de muchas calles bidireccoinales. 
Luego todos los que son 0 se tendrán que repetir en orden contrario en las aristas

In [16]:
# Crear la nueva columna según la regla
df["Direccion_calzada"] = df["Calzada__UNIDIRECCIONAL"].apply(lambda x: 1 if x == 1 else 0)

# Reubicarla justo después de Calzada_UNIDIRECCIONAL
cols = list(df.columns)

# Posición donde está Calzada_UNIDIRECCIONAL
idx = cols.index("Calzada__UNIDIRECCIONAL")

# Reconstruir el orden dejando Direccion_calzada justo después
new_order = (
    cols[:idx+1] +
    ["Direccion_calzada"] +
    cols[idx+1:-1]  # evitar duplicar la columna
)

df = df[new_order]

# Eliminamos las otras columnas
df.drop(
    columns=[
       "Calzada__BIDIRECCIONAL",
       "Calzada__BIDIRECCIONAL CON BANDEJON",
       "Calzada__UNIDIRECCIONAL"
    ],
    inplace=True
)
df.to_csv("edges_nuevo.csv", index=False)

Noté que como estamos trabajando con Santiago urbano, entonces las columnas Zona__RURAL y Zona__URBANA se van 


In [17]:
# Tambien eliminamos las columnas duplicadas
df = df.loc[:, ~df.columns.duplicated()]

df.drop(
    columns=[
       "Zona__RURAL","Zona__URBANA"
    ],
    inplace=True
)

df.to_csv("edges_nuevo.csv", index=False)

Eliminamos las columnas con:
* "Ubicación__CRUCE CON SEMAFORO APAGADO",
* "Ubicación__CRUCE CON SEMAFORO FUNCIONANDO"
Practicamente un 80% de los datos tenian en 0 estas columnas, lo que es evidencia a mi juciio de un error del dataset (no es realista)

In [ ]:
df.drop(
    columns=[
       "Ubicación__CRUCE CON SEMAFORO APAGADO",
    "Ubicación__CRUCE CON SEMAFORO FUNCIONANDO",
    
    ],
    inplace=True
)

df.to_csv("edges_nuevo.csv", index=False)

Con respecto a estas columnas:
* "Ubicación__CRUCE REGULADO POR CARABINERO", # Se elimina
* "Ubicación__CRUCE SIN SEÑALIZACION", # Tambien hay poco, casi 10% Se elimina
* "Ubicación__CRUCE CON SEÑAL CEDA EL PASO", # Hay pocos casi 20%, Se normaliza y se deja
* "Ubicación__CRUCE CON SEÑAL PARE", # Hay poco tambien casi 30%, se deja y se normaliza

In [19]:
df.drop(
    columns=[
       "Ubicación__CRUCE REGULADO POR CARABINERO",
    "Ubicación__CRUCE SIN SEÑALIZACION",
    ],
    inplace=True
)

# Acotar a 1 la columna CEDA EL PASO
df["Ubicación__CRUCE CON SEÑAL CEDA EL PASO"] = df["Ubicación__CRUCE CON SEÑAL CEDA EL PASO"].clip(upper=1)

# Renombrar a Señal_ceda_el_paso
df = df.rename(columns={
    "Ubicación__CRUCE CON SEÑAL CEDA EL PASO": "Señal_ceda_el_paso"
})

# Acotar a 1 la columna PARE
df["Ubicación__CRUCE CON SEÑAL PARE"] = df["Ubicación__CRUCE CON SEÑAL PARE"].clip(upper=1)

# Renombrar a Señal_pare
df = df.rename(columns={
    "Ubicación__CRUCE CON SEÑAL PARE": "Señal_pare"
})

df.to_csv("edges_nuevo.csv", index=False)

Columna "Ubicación__ACERA O BERMA" se elimina, casi 10% tiene valores distintos de 0, lo mismo con "Ubicación__ENLACE A DESNIVEL", "Ubicación__ENLACE A NIVEL". Y la columna "Ubicación__OTROS NO CONSIDERADOS" no se entiende.

In [ ]:
df.drop(
    columns=[
    "Ubicación__ACERA O BERMA",
    "Ubicación__ENLACE A DESNIVEL",
    "Ubicación__ENLACE A NIVEL",
    "Ubicación__OTROS NO CONSIDERADOS",
    ],
    inplace=True
)


In [22]:
df.to_csv("edges_nuevo.csv", index=False)

Se elimina tambien
* "Ubicación__PLAZA DE PEAJE": Sólo 3 unos
* "Ubicación__PUENTE", tambien

In [24]:
df.drop(
    columns=[
    "Ubicación__PLAZA DE PEAJE",
    "Ubicación__PUENTE",
    ],
    inplace=True
)

df.to_csv("edges_nuevo.csv", index=False)

Eliminamos por pocos datos:
* "Ubicación__ROTONDA",
* "Ubicación__TRAMO DE VIA CURVA HORIZONTAL",
* "Ubicación__TRAMO DE VIA CURVA VERTICAL",
Y renombramos "Ubicación__TRAMO DE VIA RECTA" por "Calle_recta" y sera 1 si la columna de antes es 1 y 0 si no lo es.

In [23]:
df.drop(
    columns=[
    "Ubicación__ROTONDA",
    "Ubicación__TRAMO DE VIA CURVA HORIZONTAL",
    "Ubicación__TRAMO DE VIA CURVA VERTICAL"
    ],
    inplace=True
)

# Convertir a 1 si es >= 1, de lo contrario 0
df["Ubicación__TRAMO DE VIA RECTA"] = (df["Ubicación__TRAMO DE VIA RECTA"] >= 1).astype(int)

# Renombrar la columna
df = df.rename(columns={
    "Ubicación__TRAMO DE VIA RECTA": "Calle_recta"
})

df.to_csv("edges_nuevo.csv", index=False)

De las siguientes columnas:
* "Ubicación__TUNEL", Se va por pocos datos
* "hw_busway", Pocos datos

* "hw_living_street", : Calle prioridad para peatones (hay casi un 10%)
* "hw_motorway", : Pocos datos
* "hw_motorway_link",: Pocos datos

* "hw_primary": Hay cerca de un 30% Carretera grande
* "hw_primary_link", : Pocos datos

* "hw_residential",: Cerca de 40% weno Calle residencial

* "hw_secondary",: Carreteras secundarias 
* "hw_secondary_link": Calles hacia carreteras secundarias. Poquisimos datos disitntos

* "hw_tertiary",: Calles o carreteras terciarias: conecta comunas (casi un 30 %)
* "hw_tertiary_link", Solo 2 datos distintos, se va

* "hw_trunk": Tres datos true, pa juera
* "hw_unclassified",: Pocos datos, se va 


In [25]:
df.drop(
    columns=[
    "Ubicación__TUNEL",
    "hw_busway",
    "hw_motorway",
    "hw_motorway_link",
    "hw_primary_link",
    "hw_secondary_link",
    "hw_tertiary_link",
    "hw_trunk",
    "hw_unclassified"
    ],
    inplace=True
)

# Renombramos las columnas
df = df.rename(columns={
    "hw_living_street": "Calle_prioridad_peatones",
    "hw_primary": "Calle_principal",
    "hw_secondary": "Calle_secundaria",
    "hw_tertiary": "Calle_terciaria",
    "hw_residential": "Calle_residencial"
})

df.to_csv("edges_nuevo.csv", index=False)

In [26]:
df = df.astype({col: 'int' for col in df.select_dtypes(include='bool').columns})
df.to_csv("edges_nuevo.csv", index=False)

Ahora definimos que el límite de la velocidad sólo este en una columna que se llame max_velocidad:
* 5 si ms_5 = 1
* 10 si ms_10 = 1
* Lo mismo para: ms_15
* ms_20
* ms_30
* ms_35
* ms_40
* ms_50
* ms_60
* ms_70
* ms_80
* ms_100
* ms_120

Y si en una misma fila aparecen dos 1's en dos columnas disitntas, entonces conserva el de menor velocidad y el otro que quede en 0.

In [27]:
# Lista de columnas de velocidad con su numerito asociado
vel_cols = {
    "ms_5": 5,
    "ms_10": 10,
    "ms_15": 15,
    "ms_20": 20,
    "ms_30": 30,
    "ms_35": 35,
    "ms_40": 40,
    "ms_50": 50,
    "ms_60": 60,
    "ms_70": 70,
    "ms_80": 80,
    "ms_100": 100,
    "ms_120": 120
}

# Crear max_velocidad tomando la menor velocidad presente en cada fila
def calcular_velocidad(row):
    velocidades = [v for col, v in vel_cols.items() if row[col] == 1]
    if len(velocidades) == 0:
        return 0  # si ninguna está en 1
    return min(velocidades)

df["max_velocidad"] = df.apply(calcular_velocidad, axis=1)


# Ajustar las columnas ms_X para que solo quede con 1 la del mínimo (si quieres)
for col, vel in vel_cols.items():
    df[col] = (df["max_velocidad"] == vel).astype(int)


# Borramos las antiguas:
vel_cols_list = [
    "ms_5","ms_10","ms_15","ms_20","ms_30","ms_35","ms_40",
    "ms_50","ms_60","ms_70","ms_80","ms_100","ms_120"
]

df = df.drop(columns=vel_cols_list)
df.to_csv("edges_nuevo.csv", index=False)

In [30]:

# Convertimos a km/h
df["Max_velocidad_kmh"] = df["max_velocidad"] * 3.6


df.to_csv("edges_nuevo.csv", index=False)

In [31]:
# Definimos la velocidad 0 como 50 km/h
df["Max_velocidad_kmh"] = df["Max_velocidad_kmh"].replace(0, 50)
df.to_csv("edges_nuevo.csv", index=False)

In [32]:
# Eliminamos las colulmnas malas
df.drop(
    columns=[
    "max_velocidad","max_velocidad_kmh"
    ],
    inplace=True
)
df.to_csv("edges_nuevo.csv", index=False)

Renombramos el largo de la calle y redondeamos su valor

In [33]:
df = df.rename(columns={"length_m": "Largo_calle"})
df["Largo_calle"] = df["Largo_calle"].round(2)
df.to_csv("edges_nuevo.csv", index=False)

Ahora juntaremos las columas en solo una que sea Tipo_calle y su valor será
* 0 si Calle_residencial = 1
* 1 si Calle_principal = 1
* 2 si Calle_secundaria = 1
* 3 si Calle_terciaria = 1
* 4 si se da otro caso (el otro caso es donde todos los anteriores es 0)

In [34]:

conditions = [
    df["Calle_residencial"] == 1,
    df["Calle_principal"] == 1,
    df["Calle_secundaria"] == 1,
    df["Calle_terciaria"] == 1
]

choices = [0, 1, 2, 3]

df["Tipo_calle"] = np.select(conditions, choices, default=4)
df.to_csv("edges_nuevo.csv", index=False)

In [35]:
df = df.drop(columns=[
    "Calle_residencial",
    "Calle_principal",
    "Calle_secundaria",
    "Calle_terciaria"
])
df.to_csv("edges_nuevo.csv", index=False)